In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action='ignore')


import torch
print(torch.__version__)
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

from tools import eval_summary
from tools import save_feature_importance

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER
True


device(type='cuda', index=0)

In [42]:
print(os.getcwd())

base_path = '.'

# df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)

df_train = joblib.load('data/df_fea_1.pkl')

/home/aiden/src/dacon_14


In [43]:
df_train

,year_month,text,smishing,morphs,morphs_str,nouns,nouns_str,fea__EC,fea__EF,fea__EP,...,fea__VA,fea__VCN,fea__VCP,fea__VV,fea__VX,fea__XPN,fea__XR,fea__XSA,fea__XSN,fea__XSV
id,,,,,,,,,,,,,,,,,,,,,
0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[XXX, 은, 행성, 산, XXX, 팀장, 입니다, ., 행복, 한, 주말, 되,...",XXX 은 행성 산 XXX 팀장 입니다 . 행복 한 주말 되 세요,"[행성, 산, 팀장, 행복, 주말]",행성 산 팀장 행복 주말,0,2,1,...,0,0,1,0,0,0,0,1,0,1
1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 도, 많이, 웃, 으시, 는, 하루, 시작, 하, 세요, XXX, 은행, ...",오늘 도 많이 웃 으시 는 하루 시작 하 세요 XXX 은행 진월동 VIP 라운지 X...,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",오늘 하루 시작 은행 진월동 라운지,1,1,2,...,0,0,0,2,0,0,0,0,0,1
2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 하, 십니까, 고객, 님, ., XXX, 은행, 입니다, ., 금일, 납부...",안녕 하 십니까 고객 님 . XXX 은행 입니다 . 금일 납부 하 셔야 할 금액 은...,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",안녕 고객 은행 금일 납부 금액 원 감사 새해 복 은행 옥포,2,5,4,...,0,0,2,2,1,0,0,0,1,3
4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[XXX, 고객, 님, 안녕, 하, 세요, XXX, 은행, XXX, 지점, 입니다,...",XXX 고객 님 안녕 하 세요 XXX 은행 XXX 지점 입니다 지난 한 해 동안 저...,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",고객 안녕 은행 지점 해 동안 저희 지점 성원 감사 시작 년 소망 일 고객 가정 건...,1,5,5,...,0,0,1,7,1,0,0,2,2,4
5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[1, 월, 은, 새로움, 이, 가득, XXX, 입니다, ., 올, 한, 해, 더,...",1 월 은 새로움 이 가득 XXX 입니다 . 올 한 해 더 많이 행복 한 한 해 되...,"[월, 한, 행복, 해]",월 한 행복 해,2,1,1,...,1,0,1,1,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341621,2019-04,(광고)XXX 고객님안녕하십니까!!고객님의 소중한 업무처리를 도와드린 kb창원중앙동...,-1,"[(, 광고, ), XXX, 고객, 님, 안녕, 하, 십니까, !, !, 고객, 님...",( 광고 ) XXX 고객 님 안녕 하 십니까 ! ! 고객 님 의 소중 한 업무 처리...,"[광고, 고객, 안녕, 고객, 업무, 처리, 창원, 중앙동, 지점, 계장, 업무, ...",광고 고객 안녕 고객 업무 처리 창원 중앙동 지점 계장 업무 처리 불편 아침 봄비 ...,7,8,8,...,2,0,1,13,1,0,1,3,3,2
341622,2019-04,고객님 항상 저희 지점을 거래해 주셔서 감사합니다 본점 서비스 설문 전화 받으시면...,-1,"[고객, 님, 항상, 저희, 지점, 을, 거래, 해, 주, 셔서, 감사, 합니다, ...",고객 님 항상 저희 지점 을 거래 해 주 셔서 감사 합니다 본점 서비스 설문 전화 ...,"[고객, 저희, 지점, 거래, 감사, 본점, 서비스, 설문, 전화, 동의, 부탁, ...",고객 저희 지점 거래 감사 본점 서비스 설문 전화 동의 부탁 오늘 하루 은행 호평,5,2,3,...,1,0,0,4,1,0,0,0,1,2
341623,2019-04,XXX 고객님 저희 XXX은행 XXX지점을 이용해주셔서 감사드립니다 이번주도 더 ...,-1,"[XXX, 고객, 님, 저희, XXX, 은행, XXX, 지점, 을, 이용, 해, 주...",XXX 고객 님 저희 XXX 은행 XXX 지점 을 이용 해 주 셔서 감사 드립니다 ...,"[고객, 저희, 은행, 지점, 이용, 감사, 이번, 주도, 행복, 시간, 기원, 은...",고객 저희 은행 지점 이용 감사 이번 주도 행복 시간 기원 은행 화정,5,1,2,...,1,0,0,3,1,0,0,1,2,2


In [3]:
train_size = len(df_train)
print(train_size)

# cat_cols = []
# fea_cols = tfidf_cols + cat_cols + cnt_cols
# fea_cols = cnt_cols + cat_cols
# fea_cols = tfidf_cols + cat_cols
# len(fea_cols)

295945


In [28]:
df_train = df_train.reset_index()

#### Params

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold




#### Train

In [21]:
fold5 = {}
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, random_state=8405)
i = 0
for train_index, valid_index in tqdm_notebook(skf.split(df_train.values, df_train['smishing'].values), total=n_splits, desc = 'CV'):
#     print(train_index)
    fold5[i] = valid_index
    i += 1



In [22]:
fold8 = {}
n_splits = 8
skf = StratifiedKFold(n_splits=n_splits, random_state=8405)
i = 0
for train_index, valid_index in tqdm_notebook(skf.split(df_train.values, df_train['smishing'].values), total=n_splits, desc = 'CV'):
#     print(train_index)
    fold8[i] = valid_index
    i += 1



In [23]:
fold16 = {}
n_splits = 16
skf = StratifiedKFold(n_splits=n_splits, random_state=8405)
i = 0
for train_index, valid_index in tqdm_notebook(skf.split(df_train.values, df_train['smishing'].values), total=n_splits, desc = 'CV'):
#     print(train_index)
    fold16[i] = valid_index
    i += 1



In [38]:
fold32 = {}
n_splits = 32
skf = StratifiedKFold(n_splits=n_splits, random_state=8405)
i = 0
for train_index, valid_index in tqdm_notebook(skf.split(df_train.values, df_train['smishing'].values), total=n_splits, desc = 'CV'):
#     print(train_index)
    fold32[i] = valid_index
    i += 1



In [15]:
len(set(fold5[4]))

59188

In [17]:
len(set(fold5[4]) & set(fold8[12]))

3701

In [18]:
len(set(fold5[4]) & set(fold8[13]))

18496

In [25]:
len(set(fold8[7]) & set(fold16[14]))

18496

In [20]:
len(set(fold5[4]) & set(fold8[15]))

18495

In [41]:
len(set(fold16[14]) & set(fold32[29]))

9246

In [37]:
df_ea = df_train.loc[fold16[14]]
for t in df_ea[df_ea['smishing'] == 1]['text'].values:
    if '광고' not in t:
        print(t, '\n')

nrB59OO147MMS SPAM Report Message정확하고 올바른 신용대출 당사상품의 자격기준과 심사기준이 변동되어 상품에 대해 고객님께 간단하게 상품정보를 전달드립니다. XXX은행 대출상품안내 1. 직장인신용대출 한도: 1억3000만원 금리: 연 3.25% 자격: XXX은행 자체등급과 기업리스트에 따라 차등적용. 2. 채무통합전환대출 한도: 연봉대비250% 금리: 연 3.45% 자격: XXX은행 자체등급과 기업리스트에 따라 차등적용. 3. 홀씨 한도: 2500만원. 금리: 연 8.68%12% 자격: 연소득 4500만원 이하 고객대상으로 XXX은행 자체등급에 따라 차등적용. XXX은행 상품의 특징 1. 당사 거래가 전혀 없어도 대출가능. 2. 시중은행보다 높은 한도. 3. 자체등급으로 판단하므로 저신용자도 대출가능. 4. 부채가 많다면 부채통합상품으로 전환가능. 5. 원리금균등방식을 이자만 납입하는 만기일시로 전환가능. 6. 가상조회를 통해 한도 및 금리를 한번에 확인가능. 이런분들께 권해드립니다. 1. 이미 시중은행권에서 대출한도를 모두 사용중이신분. 2. 기대출의 원리금균등방식으로 인해 월납입금액이 부담스러우신분. 3. 총 대출금액은 작으나 대출건수가 많아서 관리가 안되시는분. 4. 이제라도 대출로 인한 신용등급 관리 및 상승에 관심이 있으신분. 신용에 대한 기본적인 상식과 지식을 알아야 하는 시대입니다. 최소한의 지식을 본인이 알아야 금전적 손해를 보는 일이 없습니다. 신청방법:상담 또는 00시 00분 상담이라고 답장을 보내주시거나 전화를 주시면 친절하고 안전한 상담으로 도움드리겠습니다. 수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 수신거부 목록에 등록을 시켜 두번 다시 발송되지 않도록 조치를 취해드리겠습니다.무료거부 XXX-XXX-XXX제목없음 

XXX정확하고 올바른 신용대출당사상품의 자격기준과 심사기준이 완화되어 상품에 대해 고객님께 간단하게 상품정보를 전달드립니다.5분이내 빠른 대출가능여부확인 XXX 대출상품안내1. 직장인신용

#### Submit

In [ ]:
df_submit = df_test[['smishing']]
df_submit['smishing'] = 0
for csv in pred_csv:
    print(csv)
    df = pd.read_csv(csv, index_col=0)
    c = csv.split('__')[0][-1]
    df_submit['smishing_{}'.format(c)] = df['smishing']

In [ ]:
pred_cols = [c for c in df_submit.columns if 'smishing_' in c]
df_submit['std'] = df_submit[pred_cols].std(axis=1)
df_submit['median'] = df_submit[pred_cols].median(axis=1)
df_submit['mean'] = df_submit[pred_cols].median(axis=1)

pred_cols_n4 = [c for c in pred_cols if '_4' in c]
df_submit['std_n4'] = df_submit[pred_cols_n4].std(axis=1)
df_submit['median_n4'] = df_submit[pred_cols_n4].median(axis=1)
df_submit['mean_n4'] = df_submit[pred_cols_n4].median(axis=1)


#### Save

In [ ]:
print(model_ts)
df_submit['smishing'] = df_submit['median']
df_submit[['smishing']].to_csv('submit/{}_lgb_median.csv'.format(model_ts), index=True)

df_submit['smishing'] = df_submit['mean_n4']
df_submit[['smishing']].to_csv('submit/{}_lgb_n4.csv'.format(model_ts), index=True)

df_submit['smishing'] = df_submit['mean']
df_submit[['smishing']].to_csv('submit/{}_lgb.csv'.format(model_ts), index=True)

#### Eval

##### Feature Imp

In [ ]:
fea_imp_dict = {}
for pkl in glob.glob(f'model/{model_ts}*split.pkl'):
    for k, v in joblib.load(pkl).items():
        if k in fea_imp_dict.keys():
            fea_imp_dict[k] += v
        else:
            fea_imp_dict[k] = v
sorted(fea_imp_dict.items(), key=(lambda x:x[1]), reverse=True)

In [ ]:
non_zero_cols = [k for k, v in fea_imp_dict.items() if v != 0]
zero_cols = set(fea_imp_dict.keys()) - set(non_zero_cols)

print('non_zero_cols', len(non_zero_cols) , 'zero_cols', len(zero_cols))

zero_dict = {}
non_zero_dict = {}
for k, v in fea_dict.items():
    if v in zero_cols:
        zero_dict[k] = v
    if v in non_zero_cols:
        non_zero_dict[k] = v
        

joblib.dump(zero_dict, f'model/{model_ts}_zero_dict.pkl')
joblib.dump(non_zero_dict, f'model/{model_ts}_non_zero_dict.pkl')

In [ ]:
df_std = df_submit[df_submit['std'] < 1]
print(df_std.shape)
df_std.sort_values('std', ascending=False).head(30)

In [ ]:
df_submit['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
(df_submit['smishing'] * 10).astype(int).value_counts(sort=False)

In [ ]:
print(model_ts)
df_submit[['smishing']].to_csv('submit/{}_lgb.csv'.format(model_ts), index=True)